diff?

diff code blocks?


TODO: make compatible with PECL highlight and PHP code


In [ ]:
// npm install diff --save
var { JSDOM } = require('jsdom');
var JsDiff = require('diff');
const Prism = require('prismjs');
//const proxyquire = require('proxyquire');

function htmlEntities(str) {
    return str.replace(/[\u00A0-\u9999<>\&]/gim, i => '&#' + i.charCodeAt(0) + ';');
}

async function prismHighlightHtml(code, lang = 'javascript') {
    // Load the language if not already loaded
    if (!Prism.languages[lang]) {
        try {
            require(`prismjs/components/prism-${lang}.js`);
        } catch (err) {
            throw new Error(`Prism language "${lang}" could not be loaded.`);
        }
    }

    const dom = new JSDOM(`<body><pre><code class="language-${lang}">${code}</code></pre></body>`, {
        contentType: 'text/html',
    });

    const codeElement = dom.window.document.querySelector('code');

    // TODO: await import
    let ctx = {
        Prism: Prism,
        document: dom.window.document,
        window: dom.window,
        getComputedStyle: dom.window.getComputedStyle
    }
    //proxyquire('prismjs/components/prism-javascript', ctx)
    importer.import('keep-markup.js', ctx)
    //require('prismjs/components/prism-javascript');
    //require('prismjs/plugins/keep-markup/prism-keep-markup.js'); // Make sure it's installed

    Prism.highlightAllUnder(dom.window.document)
    //Prism.highlightElement(codeElement, Prism.languages[lang]); // Triggers plugin processing

    return codeElement.innerHTML;
}

async function diffTwoTexts(left, right) {
    var diff = JsDiff.diffLines(left, right);
    var code = '';
    for (var i = 0; i < diff.length; i++) {
        if (diff[i].added && diff[i + 1] && diff[i + 1].removed) {
            var swap = diff[i];
            diff[i] = diff[i + 1];
            diff[i + 1] = swap;
        }

        if (diff[i].removed) {
            code += '<span class="del">' + htmlEntities(diff[i].value) + '</span>';
        } else if (diff[i].added) {
            code += '<span class="ins">' + htmlEntities(diff[i].value) + '</span>';
        } else {
            code += '<span>' + htmlEntities(diff[i].value) + '</span>';
        }
    }
    return await prismHighlightHtml(code);
}

module.exports = {
    diffTwoTexts,
    prismHighlightHtml
}



prism-style.css?

ROUTE = /prism-style.css


In [ ]:

.token.comment,
.token.prolog,
.token.doctype,
.token.cdata {
    color: slategray;
}

.token.punctuation {
    color: #999;
}

.namespace {
    opacity: .7;
}

.token.property,
.token.tag,
.token.boolean,
.token.number,
.token.constant,
.token.symbol,
.token.deleted {
    color: #905;
}

.token.selector,
.token.attr-name,
.token.string,
.token.char,
.token.builtin,
.token.inserted {
    color: #690;
}

.token.operator,
.token.entity,
.token.url,
.language-css .token.string,
.style .token.string {
    color: #a67f59;
    background: hsla(0, 0%, 100%, .5);
}

.token.atrule,
.token.attr-value,
.token.keyword {
    color: #07a;
}

.token.function {
    color: #DD4A68;
}

.token.regex,
.token.important,
.token.variable {
    color: #e90;
}

.token.important,
.token.bold {
    font-weight: bold;
}

.token.italic {
    font-style: italic;
}

.token.entity {
    cursor: help;
}
.del {
    text-decoration: none;
    background: #fadad7;
}
.ins {
    background: #eaf2c2;
    text-decoration: none;
}



test prism js?



In [ ]:

async function testPrism() {
  var left, right;
  var r = importer.interpret(['search notebook component', 'files in project', 'diff code blocks']);
  left = r[0].code;
  var { listInProject } = importer.import('files in project');
  var { prismHighlightHtml, diffTwoTexts } = importer.import('diff code blocks');

  var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
  var project = PROFILE_PATH + '/universal';

  var components = listInProject(project, '**/search.component.ts');
  right = fs.readFileSync(components[0]).toString();
  return '<style>' + importer.interpret('prism-style.css').code + '</style>\n' + `<pre>${await diffTwoTexts(left, right)}</pre>`
}

module.exports = testPrism




keep-markup.js?

ROUTE = /keep-markup.js


In [ ]:
(function () {

	if (typeof Prism === 'undefined' || typeof document === 'undefined' || !document.createRange) {
		return;
	}

	Prism.plugins.KeepMarkup = true;

	Prism.hooks.add('before-highlight', function (env) {
		if (!env.element.children.length) {
			return;
		}

		if (!Prism.util.isActive(env.element, 'keep-markup', true)) {
			return;
		}

		var dropTokens = Prism.util.isActive(env.element, 'drop-tokens', false);
		/**
		 * Returns whether the given element should be kept.
		 *
		 * @param {HTMLElement} element
		 * @returns {boolean}
		 */
		function shouldKeep(element) {
			if (dropTokens && element.nodeName.toLowerCase() === 'span' && element.classList.contains('token')) {
				return false;
			}
			return true;
		}

		var pos = 0;
		var data = [];
		function processElement(element) {
			if (!shouldKeep(element)) {
				// don't keep this element and just process its children
				processChildren(element);
				return;
			}

			var o = {
				// Store original element so we can restore it after highlighting
				element: element,
				posOpen: pos
			};
			data.push(o);

			processChildren(element);

			o.posClose = pos;
		}
		function processChildren(element) {
			for (var i = 0, l = element.childNodes.length; i < l; i++) {
				var child = element.childNodes[i];
				if (child.nodeType === 1) { // element
					processElement(child);
				} else if (child.nodeType === 3) { // text
					pos += child.data.length;
				}
			}
		}
		processChildren(env.element);

		if (data.length) {
			// data is an array of all existing tags
			env.keepMarkup = data;
		}
	});

	Prism.hooks.add('after-highlight', function (env) {
		if (env.keepMarkup && env.keepMarkup.length) {

			var walk = function (elt, nodeState) {
				for (var i = 0, l = elt.childNodes.length; i < l; i++) {

					var child = elt.childNodes[i];

					if (child.nodeType === 1) { // element
						if (!walk(child, nodeState)) {
							return false;
						}

					} else if (child.nodeType === 3) { // text
						if (!nodeState.nodeStart && nodeState.pos + child.data.length > nodeState.node.posOpen) {
							// We found the start position
							nodeState.nodeStart = child;
							nodeState.nodeStartPos = nodeState.node.posOpen - nodeState.pos;
						}
						if (nodeState.nodeStart && nodeState.pos + child.data.length >= nodeState.node.posClose) {
							// We found the end position
							nodeState.nodeEnd = child;
							nodeState.nodeEndPos = nodeState.node.posClose - nodeState.pos;
						}

						nodeState.pos += child.data.length;
					}

					if (nodeState.nodeStart && nodeState.nodeEnd) {
						// Select the range and wrap it with the element
						var range = document.createRange();
						range.setStart(nodeState.nodeStart, nodeState.nodeStartPos);
						range.setEnd(nodeState.nodeEnd, nodeState.nodeEndPos);
						nodeState.node.element.innerHTML = '';
						nodeState.node.element.appendChild(range.extractContents());
						range.insertNode(nodeState.node.element);
						range.detach();

						// Process is over
						return false;
					}
				}
				return true;
			};

			// For each tag, we walk the DOM to reinsert it
			env.keepMarkup.forEach(function (node) {
				walk(env.element, {
					node: node,
					pos: 0
				});
			});
			// Store new highlightedCode for later hooks calls
			env.highlightedCode = env.element.innerHTML;
		}
	});
}());
